## 深入浅出pandas-4

我们再来补充一些使用`DataFrame`做数据分析时会使用到的操作，这些操作不仅常见而且也非常重要。

### 计算同比环比

我们之前讲过一个统计月度销售额的例子，我们可以通过`groupby`方法做分组聚合，也可以通过`pivot_table`生成透视表，如下所示。

In [1]:
import numpy as np
import pandas as pd
sales_df = pd.read_excel('data/2020年销售数据.xlsx')
sales_df['月份'] = sales_df.销售日期.dt.month
sales_df['销售额'] = sales_df.售价 * sales_df.销售数量
result_df = sales_df.pivot_table(index='月份', values='销售额', aggfunc='sum')
result_df.rename(columns={'销售额': '本月销售额'}, inplace=True)
result_df

,本月销售额
月份,
1,5409855
2,4608455
3,4164972
4,3996770
5,3239005
6,2817936
7,3501304
8,2948189
9,2632960


在得到月度销售额之后，如果我们需要计算月环比，这里有两种方案。第一种方案是我们可以使用`shift`方法对数据进行移动，将上一个月的数据与本月数据对齐，然后通过`(本月销售额 - 上月销售额) / 上月销售额`来计算月环比，代码如下所示。

In [2]:
result_df['上月销售额'] = result_df.本月销售额.shift(1)
result_df

,本月销售额,上月销售额
月份,,
1,5409855,NaN
2,4608455,5409855.0
3,4164972,4608455.0
4,3996770,4164972.0
5,3239005,3996770.0
6,2817936,3239005.0
7,3501304,2817936.0
8,2948189,3501304.0
9,2632960,2948189.0


在上面的例子中，`shift`方法的参数为`1`表示将数据向下移动一个单元，当然我们可以使用参数`-1`将数据向上移动一个单元。相信大家能够想到，如果我们有更多年份的数据，我们可以将参数设置为`12`，这样就可以计算今年的每个月与去年的每个月之间的同比。

In [3]:
result_df['环比'] = (result_df.本月销售额 - result_df.上月销售额) / result_df.上月销售额
result_df.style.format(
    formatter={'上月销售额': '{:.0f}', '环比': '{:.2%}'},
    na_rep='--------'
)

,本月销售额,上月销售额,环比
月份,,,
1,5409855,--------,--------
2,4608455,5409855,-14.81%
3,4164972,4608455,-9.62%
4,3996770,4164972,-4.04%
5,3239005,3996770,-18.96%
6,2817936,3239005,-13.00%
7,3501304,2817936,24.25%
8,2948189,3501304,-15.80%
9,2632960,2948189,-10.69%


> **说明**：使用 JupyterLab 时，可以通过`DataFrame`对象的`style`属性在网页中对其进行渲染，上面的代码通过`Styler`对象的`format`方法将环比格式化为百分比进行显示，此外还指定了将空值替换为`--------`。

更为简单的第二种方案是直接使用`pct_change`方法计算变化的百分比，我们先将之前的上月销售额和环比列删除掉。


In [4]:
result_df.drop(columns=['上月销售额', '环比'], inplace=True)

接下来，我们使用`DataFrame`对象的`pct_change`方法完成环比的计算。值得一提的是，`pct_change`方法有一个名为`periods`的参数，它的默认值是`1`，计算相邻两项数据变化的百分比，这不就是我们想要的环比吗？如果我们有很多年的数据，在计算时把这个参数的值修改为`12`，就可以得到相邻两年的月同比。

In [5]:
result_df['环比'] = result_df.pct_change()
result_df

,本月销售额,环比
月份,,
1,5409855,NaN
2,4608455,-0.148137
3,4164972,-0.096232
4,3996770,-0.040385
5,3239005,-0.189594
6,2817936,-0.129999
7,3501304,0.242507
8,2948189,-0.157974
9,2632960,-0.106923


我们再来看看`Index`类型，它为`Series`和`DataFrame`对象提供了索引服务，有了索引我们就可以排序数据（`sort_index`方法）、对齐数据（在运算和合并数据时非常重要）并实现对数据的快速检索（索引运算）。由于`DataFrame`类型表示的是二维数据，所以它的行和列都有索引，分别是`index`和`columns`。`Index`类型的创建的比较简单，通常给出`data`、`dtype`和`name`三个参数即可，分别表示作为索引的数据、索引的数据类型和索引的名称。由于`Index`本身也是一维的数据，索引它的方法和属性跟`Series`非常类似，你可以尝试创建一个`Index`对象，然后尝试一下之前学过的属性和方法在`Index`类型上是否生效。接下来，我们主要看看`Index`的几种子类型。

### 范围索引

范围索引是由具有单调性的整数构成的索引，我们可以通过`RangeIndex`构造器来创建范围索引，也可以通过`RangeIndex`类的类方法`from_range`来创建范围索引，代码如下所示。

In [6]:
sales_data = np.random.randint(400, 1000, 12)
index = pd.RangeIndex(1, 13, name='月份')
ser = pd.Series(data=sales_data, index=index)
ser

月份
1     408
2     548
3     847
4     755
5     669
6     764
7     607
8     537
9     703
10    476
11    799
12    598
dtype: int32

### 分类索引

分类索引是由定类尺度构成的索引。如果我们需要通过索引将数据分组，然后再进行聚合操作，分类索引就可以派上用场。分类索引还有一个名为`reorder_categories`的方法，可以给索引指定一个顺序，分组聚合的结果会按照这个指定的顺序进行呈现，代码如下所示。

In [7]:
sales_data = [6, 6, 7, 6, 8, 6]
index = pd.CategoricalIndex(
    data=['苹果', '香蕉', '苹果', '苹果', '桃子', '香蕉'],
    categories=['苹果', '香蕉', '桃子'],
    ordered=True
)
ser = pd.Series(data=sales_data, index=index)
ser

苹果    6
香蕉    6
苹果    7
苹果    6
桃子    8
香蕉    6
dtype: int64

基于索引分组数据，然后使用`sum`进行求和。

In [8]:
ser.groupby(level=0,observed = False).sum()

苹果    19
香蕉    12
桃子     8
dtype: int64

In [9]:
ser.index = index.reorder_categories(['香蕉', '桃子', '苹果'])
ser.groupby(level=0).sum()

C:\Users\jia_hua_wu\AppData\Local\Temp\ipykernel_9616\2959481822.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ser.groupby(level=0).sum()


香蕉    12
桃子     8
苹果    19
dtype: int64

### 多级索引

Pandas 中的`MultiIndex`类型用来表示层次或多级索引。可以使用`MultiIndex`类的类方法`from_arrays`、`from_product`、`from_tuples`等来创建多级索引，我们给大家举几个例子。

In [10]:
tuples = [(1, 'red'), (1, 'blue'), (2, 'red'), (2, 'blue')]
index = pd.MultiIndex.from_tuples(tuples, names=['no', 'color'])
index

MultiIndex([(1,  'red'),
            (1, 'blue'),
            (2,  'red'),
            (2, 'blue')],
           names=['no', 'color'])

In [11]:
arrays = [[1, 1, 2, 2], ['red', 'blue', 'red', 'blue']]
index = pd.MultiIndex.from_arrays(arrays, names=['no', 'color'])
index

MultiIndex([(1,  'red'),
            (1, 'blue'),
            (2,  'red'),
            (2, 'blue')],
           names=['no', 'color'])

In [12]:
sales_data = np.random.randint(1, 100, 4)
ser = pd.Series(data=sales_data, index=index)
ser

no  color
1   red      55
    blue     48
2   red      87
    blue     47
dtype: int32

In [13]:
ser.groupby('no').sum()

no
1    103
2    134
dtype: int32

In [14]:
ser.groupby(level=1).sum()

color
blue     95
red     142
dtype: int32

In [15]:
stu_ids = np.arange(1001, 1006)
semisters = ['期中', '期末']
index = pd.MultiIndex.from_product((stu_ids, semisters), names=['学号', '学期'])
courses = ['语文', '数学', '英语']
scores = np.random.randint(60, 101, (10, 3))
df = pd.DataFrame(data=scores, columns=courses, index=index)
df

语文  数学  英语
学号   学期            
1001 期中  65  65  89
     期末  85  84  75
1002 期中  97  80  67
     期末  82  66  77
1003 期中  97  60  74
     期末  95  61  70
1004 期中  79  91  78
     期末  81  77  63
1005 期中  76  96  85
     期末  89  66  66


根据第一级索引分组数据，按照期中成绩占`25%`，期末成绩占`75%` 的方式计算每个学生每门课的成绩。

In [16]:
df.groupby(level=0).agg(lambda x: x.values[0] * 0.25 + x.values[1] * 0.75)

,语文,数学,英语
学号,,,
1001,80.00,79.25,78.50
1002,85.75,69.50,74.50
1003,95.50,60.75,71.00
1004,80.50,80.50,66.75
1005,85.75,73.50,70.75


### 间隔索引

间隔索引顾名思义是使用固定的间隔范围充当索引，我们通常会使用`interval_range`函数来创建间隔索引，代码如下所示。

In [17]:
index = pd.interval_range(start=0, end=5)
index

IntervalIndex([(0, 1], (1, 2], (2, 3], (3, 4], (4, 5]], dtype='interval[int64, right]')

In [18]:
index.contains(1.5)

array([False,  True, False, False, False])

In [19]:
index.overlaps(pd.Interval(1.5, 3.5))

array([False,  True,  True,  True, False])

如果希望间隔范围是左闭右开的状态，可以在创建间隔索引时通过`closed='left'`来做到；如果希望两边都是关闭状态，可以将`close`参数的值赋值为`both`，代码如下所示。

### 日期时间索引

`DatetimeIndex`应该是众多索引中最复杂最重要的一种索引，我们通常会使用`date_range()`函数来创建日期时间索引，该函数有几个非常重要的参数`start`、`end`、`periods`、`freq`、`tz`，分别代表起始日期时间、结束日期时间、生成周期、采样频率和时区。我们先来看看如何创建`DatetimeIndex`对象，再来讨论它的相关运算和操作，代码如下所示。

In [20]:
pd.date_range('2021-1-1', '2021-6-30', periods=10)

DatetimeIndex(['2021-01-01', '2021-01-21', '2021-02-10', '2021-03-02',
               '2021-03-22', '2021-04-11', '2021-05-01', '2021-05-21',
               '2021-06-10', '2021-06-30'],
              dtype='datetime64[ns]', freq=None)

In [21]:
pd.date_range('2021-1-1', '2021-6-30', freq='W')

DatetimeIndex(['2021-01-03', '2021-01-10', '2021-01-17', '2021-01-24',
               '2021-01-31', '2021-02-07', '2021-02-14', '2021-02-21',
               '2021-02-28', '2021-03-07', '2021-03-14', '2021-03-21',
               '2021-03-28', '2021-04-04', '2021-04-11', '2021-04-18',
               '2021-04-25', '2021-05-02', '2021-05-09', '2021-05-16',
               '2021-05-23', '2021-05-30', '2021-06-06', '2021-06-13',
               '2021-06-20', '2021-06-27'],
              dtype='datetime64[ns]', freq='W-SUN')

> **说明**：`freq=W`表示采样周期为一周，它会默认星期日是一周的开始；如果你希望星期一表示一周的开始，你可以将其修改为`freq=W-MON`；你也可以试着将该参数的值修改为`12H`，`M`，`Q`等，看看会发生什么，相信你不难猜到它们的含义。

`DatatimeIndex`可以跟`DateOffset`类型进行运算，这一点很好理解，以为我们可以设置一个时间差让时间向前或向后偏移，具体的操作如下所示。

In [22]:
index = pd.date_range('2021-1-1', '2021-6-30', freq='W')
index - pd.DateOffset(days=2)

DatetimeIndex(['2021-01-01', '2021-01-08', '2021-01-15', '2021-01-22',
               '2021-01-29', '2021-02-05', '2021-02-12', '2021-02-19',
               '2021-02-26', '2021-03-05', '2021-03-12', '2021-03-19',
               '2021-03-26', '2021-04-02', '2021-04-09', '2021-04-16',
               '2021-04-23', '2021-04-30', '2021-05-07', '2021-05-14',
               '2021-05-21', '2021-05-28', '2021-06-04', '2021-06-11',
               '2021-06-18', '2021-06-25'],
              dtype='datetime64[ns]', freq=None)

In [23]:
index + pd.DateOffset(hours=2, minutes=10)

DatetimeIndex(['2021-01-03 02:10:00', '2021-01-10 02:10:00',
               '2021-01-17 02:10:00', '2021-01-24 02:10:00',
               '2021-01-31 02:10:00', '2021-02-07 02:10:00',
               '2021-02-14 02:10:00', '2021-02-21 02:10:00',
               '2021-02-28 02:10:00', '2021-03-07 02:10:00',
               '2021-03-14 02:10:00', '2021-03-21 02:10:00',
               '2021-03-28 02:10:00', '2021-04-04 02:10:00',
               '2021-04-11 02:10:00', '2021-04-18 02:10:00',
               '2021-04-25 02:10:00', '2021-05-02 02:10:00',
               '2021-05-09 02:10:00', '2021-05-16 02:10:00',
               '2021-05-23 02:10:00', '2021-05-30 02:10:00',
               '2021-06-06 02:10:00', '2021-06-13 02:10:00',
               '2021-06-20 02:10:00', '2021-06-27 02:10:00'],
              dtype='datetime64[ns]', freq=None)